In [15]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np

In [24]:
def calculate_metrics(data):
    # Data Exploration and Cleaning
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.dropna(inplace=True)

    # Feature Engineering
    data['position'] = data['side'] + '_' + data['positionSide']
    grouped = data.groupby('Port_ID')

    # Initialize results dictionary
    results = []

    for port_id, group in grouped:
        # Aggregated metrics
        total_investment = group['quantity'].sum()
        net_profit = group['realizedProfit'].sum()
        positive_trades = group[group['realizedProfit'] > 0]
        win_positions = len(positive_trades)
        total_positions = len(group)
        
        # Financial metrics
        roi = (net_profit / total_investment) * 100 if total_investment > 0 else 0
        win_rate = (win_positions / total_positions) * 100 if total_positions > 0 else 0
        
        # Sharpe Ratio
        daily_returns = group['realizedProfit'] / group['quantity']
        mean_return = daily_returns.mean()
        std_dev_return = daily_returns.std()
        sharpe_ratio = (mean_return / std_dev_return) if std_dev_return > 0 else 0
        
        # Maximum Drawdown (MDD)
        cumulative_profit = group['realizedProfit'].cumsum()
        rolling_max = cumulative_profit.cummax()
        drawdown = rolling_max - cumulative_profit
        max_drawdown = drawdown.max()

        # Append metrics
        results.append({
            'Port_ID': port_id,
            'ROI': roi,
            'PnL': net_profit,
            'Sharpe_Ratio': sharpe_ratio,
            'MDD': max_drawdown,
            'Win_Rate': win_rate,
            'Win_Positions': win_positions,
            'Total_Positions': total_positions
        })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Ranking Algorithm
    results_df['Score'] = (
        results_df['ROI'] * 0.4 +
        results_df['Sharpe_Ratio'] * 0.3 +
        results_df['Win_Rate'] * 0.2 -
        results_df['MDD'] * 0.1
    )
    results_df = results_df.sort_values(by='Score', ascending=False)

    return results_df


In [25]:
# Load the dataset
data = pd.read_csv('TRADES_CopyTr_90D_ROI.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'TRADES_CopyTr_90D_ROI.csv'

NameError: name 'grouped' is not defined